In [7]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as BS
import jieba
import pymysql
import re

In [56]:
#连接数据库
conn = pymysql.connect('localhost','root','9432','companyb',charset='utf8')
cursor=conn.cursor()

In [3]:
cursor.execute("SELECT indi,count(indi) FROM (select indi from `com_inv_rel_1` GROUP BY(investor)) as b  GROUP BY(indi)")
a=cursor.fetchall()

In [35]:
cursor.execute("SELECT industry,count(industry) FROM (select industry from `com_inv_rel_1` GROUP BY(company)) as b GROUP BY(industry)")
c=cursor.fetchall()

In [57]:
cursor.execute("SELECT indi,industry FROM `com_inv_rel_1` where indi is not null and industry is not null")
e=cursor.fetchall()

In [142]:
g=pd.read_csv('省份投资机构数.csv')
g.columns=['loc','count']

In [124]:
cursor.execute("select indi,industry,count(*) from com_inv_rel_1 where indi is not null and industry is not null GROUP BY indi,industry ORDER BY count(*) desc limit 0,2000")
h=cursor.fetchall()

In [10]:
b=[]
for i in range(len(a)):
    if a[i][1]>3:
        b.append(a[i])
print(len(b))

93


In [39]:
d=[]
for i in range(len(c)):
    if c[i][1]>3:
        d.append(c[i])
print(len(d))

93


In [49]:
df=pd.DataFrame(np.array(b)[:,1].reshape(1,93),columns=np.array(b)[:,0])
for i in range(len(d)):
    df.loc[1,d[i][0]]=d[i][1]

In [50]:
df

,商务服务业,批发业,房地产业,科技推广和应用服务业,零售业,软件和信息技术服务业,资本市场服务,研究和试验发展,房屋建筑业,专业技术服务业,...,金属制品、机械和设备修理业,社会工作,邮政业,国家机构,烟草制品业,石油和天然气开采业,管道运输业,群众团体、社会团体和其他成员组织,社会保障,中国共产党机关
0,72976,42243,21592,12172,10129,10008,6737,5572,5503,4614,...,113,74,67,63,30,23,13,10,6,5
1,76547,55797,38883,22187,17960,20603,5563,12462,4534,8828,...,155,454,103,18,55,48,24,15,19,4


In [17]:
b=sorted(b,key=lambda k:k[1],reverse=True)

In [40]:
d0=np.array(d)[:,0]
d1=np.array(d)[:,1]

In [19]:
b0=np.array(b)[:,0]
b1=np.array(b)[:,1]

In [146]:
from pyecharts import Bar

bar = Bar("投资机构行业分布")
bar.add("投资机构所属行业",df.columns,df.loc[0,:])
bar.add("被投资企业所属行业",df.columns,df.loc[1,:],is_more_utils=True, xaxis_interval=0, xaxis_rotate=30,is_datazoom_show=True)
bar

In [80]:
f=[]
for x in e:
    if x[0] in b0 and x[1] in b0:
        f.append(x)

In [75]:
len(f)

364359

In [81]:
df2=pd.DataFrame(np.array(f),columns=['s','t'])

In [138]:
from pyecharts import Graph
nodes = [{"name": b[i][0], "symbolSize": 10} for i in range(len(b))]
links = [{"source": x[0], "target": x[1],"value":x[2]} for x in h[:100]]
graph = Graph("关系图-力引导布局示例", width=1200, height=600)
graph.add("", nodes, links, label_pos="right",
          graph_repulsion=50, is_legend_show=False,is_label_show=True,
          line_curve=0.2,graph_edge_symbol=['circle', 'arrow'])
graph

In [149]:
allnum=0
tnum=0
for x in h:
    allnum+=x[2]
    if x[0]==x[1]:
        tnum+=x[2]
print(tnum,allnum,tnum/allnum)

116621 465438 0.2505618363777775


In [151]:
from pyecharts import Map

value = [x/400 for x in g.loc[:,'count'].tolist()]
attr =[]
for x in g.loc[:,'loc'].tolist():
    if len(x)>4 and x!='内蒙古自治区':
        attr.append(x[:2])
    elif x=='内蒙古自治区':
        attr.append('内蒙古')
    else:
        attr.append(x[:-1])
map = Map("投资机构地域分布")
map.add("", attr, value, maptype='china', is_visualmap=True,
        visual_text_color='#000')
map

In [153]:
bar = Bar("投资机构地域分布")
value = [x for x in g.loc[:,'count'].tolist()]
attr = [x for x in g.loc[:,'loc'].tolist()]
bar.add("地域",attr,value,is_more_utils=True, xaxis_interval=0, xaxis_rotate=30,is_datazoom_show=False)
bar

In [113]:
g

()